In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# XGBoost on Otto dataset, Tune n_estimators
from pandas import read_csv
import os
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import matplotlib
#matplotlib.use('Agg')
from matplotlib import pyplot
import time
import numpy as np
# load data

#df_basedata_train_0 = pd.read_csv(r'C:\Vibhaas\Artist of Analytics\Algorithms\XGBoost\Otto_train.csv')
df_basedata_train_0 = pd.read_csv('/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df_basedata_train_0.head()

In [ ]:
df_basedata_train_0.shape

Considerable size to tryout Classication problem.

In [ ]:
df_basedata_train_0.info()

In [ ]:
# Converting Total Charges to a numerical data type.
df_basedata_train_0.TotalCharges = pd.to_numeric(df_basedata_train_0.TotalCharges, errors='coerce')
df_basedata_train_0.info()


In [ ]:
# Find Categorical Variables
s = (df_basedata_train_0.dtypes == 'object')
object_cols = list(s[s].index)
print("Categorical variables:")
print(object_cols)

In [ ]:
df_1 = pd.get_dummies(df_basedata_train_0, columns=["gender","Partner","Dependents", "PhoneService", "MultipleLines", "InternetService","OnlineSecurity","OnlineBackup","DeviceProtection", "TechSupport","StreamingTV","StreamingMovies","Contract","PaperlessBilling","PaymentMethod", "Churn"   ],drop_first=True)
df_1.head()

In [ ]:
df_1.isnull().sum()

In [ ]:
#Removing missing values 
df_1.dropna(inplace = True)

In [ ]:
df_1.isnull().sum()

In [ ]:
df_1 = df_1.rename(columns={'Churn_Yes': 'target'})

## Drop ID Features
df_2=df_1.drop(['customerID'],axis=1)
df_2.head()

**EDA**

In [ ]:

corr = df_2.corr().tail(1)
corr.sort_values(by='target',axis=1)

#import for visualization
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
fig, ax = plt.subplots(figsize=(20,5))

plt.title("Correlation Plot")
plt.xlabel("Independent Vairables")
plt.ylabel("Correlation with Dependent Variable")
plt.xticks(rotation=90)

ax = sns.barplot(data=corr.sort_values(by='target',axis=1))


#import for visualization
import seaborn as sns
f,ax = plt.subplots(figsize=(35, 1))
sns.heatmap(df_2.corr().tail(1).sort_values(by='target',axis=1), annot=True, linewidths=0.5, fmt='.2f',ax=ax)

In [ ]:
# Machine Learning 

X = df_2.drop("target", axis=1)
y = df_2["target"]

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=50, max_features='sqrt')
clf = clf.fit(X, y)

#have a look at the importance of each feature.
features = pd.DataFrame()
features['feature'] = X.columns
features['importance'] = clf.feature_importances_
features.sort_values(by=['importance'], ascending=True, inplace=True)
features.set_index('feature', inplace=True)

features.plot(kind='barh', figsize=(10, 10))

In [ ]:
# Standardize
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
scale.fit(X)
predictor_standard = scale.transform(X)


In [ ]:
from sklearn.model_selection import train_test_split
# Split data to 80% training data and 20% of test to check the accuracy of our model
X_train, X_test, y_train, y_test = train_test_split(predictor_standard, y, test_size=0.20, random_state=0)



In [ ]:
#Model

from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import time


In [ ]:
#create an array of models
models = []
models.append(("LR",LogisticRegression()))
models.append(("NB",GaussianNB()))
models.append(("RF",RandomForestClassifier()))
models.append(("SVC",SVC()))
models.append(("Dtree",DecisionTreeClassifier()))
models.append(("XGB",xgb.XGBClassifier()))
models.append(("KNN",KNeighborsClassifier()))

result = []
#measure the accuracy 
for name,model in models:
    start_time = time.time()
    kfold = KFold(n_splits=5, random_state=22)
    cv_result = cross_val_score(model,X_train,y_train, cv = kfold,scoring = "accuracy")
    print(name, cv_result)
    print("-"*5,name, " Mean accuracy of cross-validation: ", format(round(cv_result.mean(),4)))
    execution_time = (time.time() - start_time)
    result.append((name,cv_result.mean(), execution_time ))
    print(name,"--- %s seconds ---" % (time.time() - start_time))

pd.DataFrame(result)
    
df = pd.DataFrame(result, columns =['Algo', 'Accuracy', 'execution_time']) 
df 




In [ ]:
# sort df by Count column
pd_df = df.sort_values(['Accuracy']).reset_index(drop=True)
print (pd_df)

In [ ]:

plt.figure(figsize=(8,8))
# plot bar chart with index as x values
ax = sns.barplot(pd_df.index, pd_df.Accuracy)
ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
ax.set(xlabel="Algo", ylabel="Acurracy")
# add proper Dim values as x labels

ax.set_xticklabels(pd_df.Algo)
for item in ax.get_xticklabels(): item.set_rotation(90)
for i, v in enumerate(pd_df["Accuracy"].iteritems()):        
    ax.text(i ,v[1], "{:,}".format(v[1]), color='m', va ='bottom', rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.model_selection import train_test_split
# Split data to 80% training data and 20% of test to check the accuracy of our model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)


#Model

from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import time


#create an array of models
models = []
models.append(("LR",LogisticRegression()))
models.append(("NB",GaussianNB()))
models.append(("RF",RandomForestClassifier()))
models.append(("SVC",SVC()))
models.append(("Dtree",DecisionTreeClassifier()))
models.append(("XGB",xgb.XGBClassifier()))
models.append(("KNN",KNeighborsClassifier()))

result = []
#measure the accuracy 
for name,model in models:
    start_time = time.time()
    kfold = KFold(n_splits=5, random_state=22)
    cv_result = cross_val_score(model,X_train,y_train, cv = kfold,scoring = "accuracy")
    print(name, cv_result)
    print("-"*5,name, " Mean accuracy of cross-validation: ", format(round(cv_result.mean(),4)))
    execution_time = (time.time() - start_time)
    result.append((name,cv_result.mean(), execution_time ))
    print(name,"--- %s seconds ---" % (time.time() - start_time))

pd.DataFrame(result)
    
df = pd.DataFrame(result, columns =['Algo', 'Accuracy', 'execution_time']) 
df 


# sort df by Count column
pd_df = df.sort_values(['Accuracy']).reset_index(drop=True)
pd_df

In [ ]:
plt.figure(figsize=(8,8))
# plot bar chart with index as x values
ax = sns.barplot(pd_df.index, pd_df.Accuracy)
ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
ax.set(xlabel="Algo", ylabel="Acurracy")
# add proper Dim values as x labels

ax.set_xticklabels(pd_df.Algo)
for item in ax.get_xticklabels(): item.set_rotation(90)
for i, v in enumerate(pd_df["Accuracy"].iteritems()):        
    ax.text(i ,v[1], "{:,}".format(v[1]), color='m', va ='bottom', rotation=45)
plt.tight_layout()
plt.show()


**Attempt 3 XGBoost**

In [ ]:
from sklearn.model_selection import train_test_split
# Split data to 80% training data and 20% of test to check the accuracy of our model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)


In [ ]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [ ]:
import xgboost as xgb
xg_clas = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 10)

In [ ]:
xg_clas.fit(X_train,y_train)

y_pred = xg_clas.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, y_pred)
print("Accuracy is " + str(round((score*100), 2))+"%")

In [ ]:
import matplotlib.pyplot as plt

xgb.plot_tree(xg_clas,num_trees=0)
plt.rcParams['figure.figsize'] = [10, 10]
plt.show()

In [ ]:
params = {"objective":"reg:logistic",'colsample_bytree': 0.3,'learning_rate': 0.1,  'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3, num_boost_round=50,early_stopping_rounds=10, as_pandas=True, seed=123)

In [ ]:
cv_results.head()


In [ ]:
import matplotlib.pyplot as plt

xgb.plot_tree(xg_reg,num_trees=0)
plt.rcParams['figure.figsize'] = [50, 50]
plt.show()

In [ ]:
from xgboost import plot_tree
import matplotlib.pyplot as plt

# plot single tree
plot_tree(xg_reg)
plt.show()

In [ ]:
xgb.plot_importance(xg_clas)
plt.rcParams['figure.figsize'] = [10, 10]

    
plt.show()

In [ ]:
for x in range(9):
    xgb.plot_tree(xg_clas,num_trees=x)
    #plt.rcParams['figure.figsize'] = [10, 10]
    plt.show()